In [ ]:

# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split
train_data_gen = ImageDataGenerator(
        rescale=1./255 )

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'drowsiness_detector/cropped_data1',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',
    shuffle=True # Set shuffle to True initially
)

# create model structure
vehicule_model = Sequential()

vehicule_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
vehicule_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
vehicule_model.add(MaxPooling2D(pool_size=(2, 2)))
vehicule_model.add(Dropout(0.25))

vehicule_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
vehicule_model.add(MaxPooling2D(pool_size=(2, 2)))
vehicule_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
vehicule_model.add(MaxPooling2D(pool_size=(2, 2)))
vehicule_model.add(Dropout(0.25))

vehicule_model.add(Flatten())
vehicule_model.add(Dense(1024, activation='relu'))
vehicule_model.add(Dropout(0.5))
vehicule_model.add(Dense(5, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

vehicule_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [18]:

# Train the neural network/model
vehicule_model_info = vehicule_model.fit(
        train_generator,
        steps_per_epoch=5914// 64,
        epochs=20,
        validation_data=train_generator ,
        validation_split=.2 ,

        shuffle=True)

Epoch 1/20
92/92 [==============================] - 45s 450ms/step - loss: 0.7859 - accuracy: 0.7031 - val_loss: 0.3852 - val_accuracy: 0.8510
Epoch 2/20
92/92 [==============================] - 42s 455ms/step - loss: 0.3523 - accuracy: 0.8550 - val_loss: 0.2415 - val_accuracy: 0.9006
Epoch 3/20
92/92 [==============================] - 37s 405ms/step - loss: 0.2815 - accuracy: 0.8855 - val_loss: 0.2583 - val_accuracy: 0.8979
Epoch 4/20
92/92 [==============================] - 40s 432ms/step - loss: 0.2355 - accuracy: 0.9022 - val_loss: 0.1781 - val_accuracy: 0.9312
Epoch 5/20
92/92 [==============================] - 42s 455ms/step - loss: 0.2104 - accuracy: 0.9147 - val_loss: 0.1952 - val_accuracy: 0.9153
Epoch 6/20
92/92 [==============================] - 40s 438ms/step - loss: 0.1953 - accuracy: 0.9241 - val_loss: 0.1305 - val_accuracy: 0.9530
Epoch 7/20
92/92 [==============================] - 39s 420ms/step - loss: 0.1600 - accuracy: 0.9390 - val_loss: 0.1038 - val_accuracy: 0.9616

In [19]:
# save model structure in jason file
model_json = vehicule_model.to_json()
with open("facefin2_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
vehicule_model.save_weights('facefin2_model.h5')